A simple Travelling Salesman Problem to solve (optimize) with IBM Cplex for learning purposes

_"Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?"_

*(Wikipedia.org, 2023)* - https://en.wikipedia.org/wiki/Travelling_salesman_problem

![problem-studied](./documentation/problem.png)

# BEFORE CODING: INSTALL CPLEX AND SETUP YOUR PYTHON ENVIRONEMENT
0. Make sure you have Python **3.7** or **3.8** installed (_Cplex currently do not support newer versions!_)
1. Download Cplex from **IBM official website** and install it: https://www.ibm.com/products/ilog-cplex-optimization-studio/cplex-optimizer
2. Go to the `CPLEX_HOME/cplex/python/3.8/x86-64_osx/cplex/` (_choose the right python version and OS platform here!_) sub-folder inside the Cplex install folder (`/Applications/CPLEX_Studio210` in MacOS) and run: `python setup.py install --home PATH_TO_PYTHON_PACKAGES/cplex` or `python3.8 setup.py install --home PATH_TO_PYTHON_PACKAGES/cplex` if you have several versions of python installed

--> _for example:_ **PATH_TO_PYTHON_PACKAGES**=`/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages`

3. **Alternative:** Install cplex using `pip install cplex dcplex` or `pip3.8 install cplex dcplex` if you have several versions of cplex installed

In [30]:
# 0.A IMPORT ALL USED LIBRARIES
from random import randint
import numpy as np
#from docplex.mp.model import Model
import cplex
from cplex import Cplex

In [31]:
# 0.B CONFIGURE THE PROBLEM
NBR_CITIES = 5
MIN_DISTANCE_BETWEEN_CITIES = 1
MAX_DISTANCE_BETWEEN_CITIES = 15

# 0.C CONFIGURE CPLEX
COMPUTING_TIME_LIMIT = 300 #seconds
MEMORY_LIMIT = 1024 #Gb
NODE_LIMIT = 1000 #size of the branch&bound/cut decision tree
MIP_TOLERANCE = 0 #%
MAX_NBR_THREADS = 4

In [32]:
# I. BUILD A RANDOM TSP INSTANCE 
def build_random_instance(nbr_cities=6, min_distance=1, max_distance=15):
    start = randint(0, nbr_cities-1) # 1. random generate the starting city
    distances = np.random.uniform(min_distance, max_distance, (nbr_cities, nbr_cities)).astype(int) # 2. random generate a matrix of (integer) paths between cities
    distances = np.triu(distances, k=1)
    distances += distances.T # 3. the distance between city a -> city b should be equal to the between city b -> city a
    np.fill_diagonal(distances, 0) # 4. replace all distance for a city to itself (the diagonal) to 0 
    return distances, start

distances, start = build_random_instance(NBR_CITIES, MIN_DISTANCE_BETWEEN_CITIES, MAX_DISTANCE_BETWEEN_CITIES)
print(distances)
print(start)

[[ 0 11  8 13 10]
 [11  0 14  7 12]
 [ 8 14  0  4  9]
 [13  7  4  0 10]
 [10 12  9 10  0]]
1


In [36]:
# II. BUILD A CPLEX MODEL WITH DECISIONS VARIABLES, THE FIRST TWO CONSTRAINTS, AND A OBJECTIVE FUNCTION
def build_model(distances, nbr_cities=6):
    m = Cplex()

    # 1. Create an array of variables with names and lower/upper bounds
    names = [f"path_{i}_{j}" for i in range(nbr_cities) for j in range(nbr_cities) if j != i]
    print(names)
    types = [m.variables.type.binary] * len(names)
    lb = [0] * len(names)
    up = [1] * len(names)
    m.variables.add(names=names, types=types, lb=lb, ub=up) 

    # 2. Create the min objective function
    for i in range(nbr_cities-1):
        for j in range(i+1, nbr_cities):
            m.objective.set_linear(f"path_{i}_{j}", int(distances[i][j]))
            m.objective.set_linear(f"path_{j}_{i}", int(distances[j][i]))
    m.objective.set_sense(m.objective.sense.minimize)

    # 3. Create the first two constraint: exactly on selected path FROM and TO each city
    for i in range(nbr_cities):
        paths_from_i = [f"path_{i}_{j}" for j in range(nbr_cities) if j != i]
        paths_to_i = [f"path_{j}_{i}" for j in range(nbr_cities) if j != i]
        m.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=paths_from_i, val=[1]*len(paths_from_i))], senses=['E'], rhs=[1])
        m.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=paths_to_i, val=[1]*len(paths_from_i))], senses=['E'], rhs=[1])
    
    # 4. Apply the CPLEX runtime configuration
    m.parameters.timelimit.set(COMPUTING_TIME_LIMIT)
    m.parameters.workmem.set(MEMORY_LIMIT)
    m.parameters.mip.limits.nodes.set(NODE_LIMIT)
    m.parameters.mip.tolerances.mipgap.set(MIP_TOLERANCE)
    m.parameters.threads.set(MAX_NBR_THREADS)
    return m

model = build_model(distances, NBR_CITIES)

['path_0_1', 'path_0_2', 'path_0_3', 'path_0_4', 'path_1_0', 'path_1_2', 'path_1_3', 'path_1_4', 'path_2_0', 'path_2_1', 'path_2_3', 'path_2_4', 'path_3_0', 'path_3_1', 'path_3_2', 'path_3_4', 'path_4_0', 'path_4_1', 'path_4_2', 'path_4_3']


![calcul-of-subsets](./documentation/calcul.png)

In [ ]:
# III. ...